In [ ]:
! pip install -U tushare
! pip install -U pandas

In [36]:
import datetime
import tushare as ts
import pandas as pd
import numpy as np


In [115]:
# 获取最近若干天的信息
end_date = datetime.datetime.today()
start_date = end_date - datetime.timedelta(days=10)
temp = start_date
df_tick = pd.DataFrame(columns=["time", "price", "change", "volume", "amount", "type"])
while temp <= end_date:
    date_str = temp.strftime("%Y-%m-%d")
    temp_data = ts.get_tick_data('002372', date=date_str, src='tt')
    if temp_data is not None:
        temp_data["time"] = date_str + " " + temp_data["time"]
        df_tick = df_tick.append(temp_data)
       
    temp = temp + datetime.timedelta(days=1)

print(df_tick.shape, df_tick.head())
# df_tick.to_csv("002372_test.csv")

(21680, 6)                   time  price  change volume  amount type
0  2020-10-19 09:25:02  16.84    0.24    263  442892   卖盘
1  2020-10-19 09:30:01  16.85    0.01     89  149931   买盘
2  2020-10-19 09:30:05  16.84   -0.01    176  295781   卖盘
3  2020-10-19 09:30:07  16.82   -0.02     85  142843  中性盘
4  2020-10-19 09:30:10  16.82    0.00    111  186312  中性盘


In [108]:
# 重设index，设置为time列，并且类型转化为datetime
df_tick.index = pd.to_datetime(df_tick.time)
df_tick.drop(axis=1, columns='time', inplace=True)
print(df_tick.head(), df_tick.tail())

                     price  change volume  amount type
time                                                  
2020-10-19 09:25:02  16.84    0.24    263  442892   卖盘
2020-10-19 09:30:01  16.85    0.01     89  149931   买盘
2020-10-19 09:30:05  16.84   -0.01    176  295781   卖盘
2020-10-19 09:30:07  16.82   -0.02     85  142843  中性盘
2020-10-19 09:30:10  16.82    0.00    111  186312  中性盘                      price  change volume   amount type
time                                                   
2020-10-28 14:56:43  16.50    0.01      7    11550   买盘
2020-10-28 14:56:52  16.49   -0.01     43    70909   卖盘
2020-10-28 14:56:58  16.48   -0.01      6     9890   卖盘
2020-10-28 14:57:01  16.51    0.03      1     1651   买盘
2020-10-28 15:00:05  16.51    0.00    653  1078103   卖盘


In [109]:
# 按照一分钟 重新采样
df_min_ohlc = df_tick.price.resample("1min", closed='left', label='left').ohlc()
print(df_min_ohlc)

                      open   high    low  close
time                                           
2020-10-19 09:25:00  16.84  16.84  16.84  16.84
2020-10-19 09:26:00    NaN    NaN    NaN    NaN
2020-10-19 09:27:00    NaN    NaN    NaN    NaN
2020-10-19 09:28:00    NaN    NaN    NaN    NaN
2020-10-19 09:29:00    NaN    NaN    NaN    NaN
...                    ...    ...    ...    ...
2020-10-28 14:56:00  16.48  16.50  16.47  16.48
2020-10-28 14:57:00  16.51  16.51  16.51  16.51
2020-10-28 14:58:00    NaN    NaN    NaN    NaN
2020-10-28 14:59:00    NaN    NaN    NaN    NaN
2020-10-28 15:00:00  16.51  16.51  16.51  16.51

[13296 rows x 4 columns]


In [110]:
# 清洗数据，非交易时间段的NaN数据
df_min_ohlc = df_min_ohlc.dropna(axis=0, how='all')
print(df_min_ohlc.head())

                      open   high    low  close
time                                           
2020-10-19 09:25:00  16.84  16.84  16.84  16.84
2020-10-19 09:30:00  16.85  16.85  16.63  16.70
2020-10-19 09:31:00  16.69  16.77  16.69  16.73
2020-10-19 09:32:00  16.73  16.76  16.72  16.76
2020-10-19 09:33:00  16.76  16.78  16.71  16.73


In [111]:
# 数据可视化 画图
! pip install -U matplotlib
! pip install -U mpl-finance
! pip install -U mplfinance


Requirement already up-to-date: matplotlib in d:\code\stock-trade-system\.env\lib\site-packages (3.3.2)

You should consider upgrading via the 'python -m pip install --upgrade pip' command.



Requirement already up-to-date: mpl-finance in d:\code\stock-trade-system\.env\lib\site-packages (0.10.1)

You should consider upgrading via the 'python -m pip install --upgrade pip' command.



Requirement already up-to-date: mplfinance in d:\code\stock-trade-system\.env\lib\site-packages (0.12.7a0)


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [112]:
series_data ={'Open': df_min_ohlc.open, 'Close': df_min_ohlc.close, 'High': df_min_ohlc.high, 'Low': df_min_ohlc.low}
df_stockload = pd.DataFrame(series_data)

In [114]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import mplfinance as mpf

# mpf支持的style
print(mpf.available_styles())

# 解决中文乱码
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['axes.unicode_minus'] = False

# 创建fig对象
# fig = plt.figure(figsize=(12,6), dpi=100, facecolor='white')
mc = mpf.make_marketcolors(up='r',down='g')
s = mpf.make_mpf_style(marketcolors=mc)
fig = mpf.figure(figsize=(12,6), dpi=1000, facecolor='white', style=s)

# 指定子图放置的网格几何位置
gs = gridspec.GridSpec(3, 1, left=0.08, bottom=0.15, right=0.99, top=0.96, wspace=None, hspace=0, height_ratios=[3.5, 1, 1])

# 添加子图
graph_KAV = fig.add_subplot(gs[0, :])
graph_MACD = fig.add_subplot(gs[1, :])
graph_KDJ = fig.add_subplot(gs[2, :])


# 绘制K线图
mpf.plot(df_stockload, ax=graph_KAV,type="candle", mav=(20,30,60))  # mav表示移动平均线



# 绘制移动平均线图
# df_stockload["Ma20"] = df_stockload.Close.rolling(window=20).mean()
# df_stockload["Ma30"] = df_stockload.Close.rolling(window=30).mean()
# df_stockload["Ma60"] = df_stockload.Close.rolling(window=60).mean()

# graph_KAV.plot(np.arange(0, len(df_stockload.index)), df_stockload['Ma20'], 'black', label='M20', lw=1.0)

plt.show()

['binance', 'blueskies', 'brasil', 'charles', 'checkers', 'classic', 'default', 'mike', 'nightclouds', 'sas', 'starsandstripes', 'yahoo']


No handles with labels found to put in legend.
d:\code\stock-trade-system\.env\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 20255 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\code\stock-trade-system\.env\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 26143 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\code\stock-trade-system\.env\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 26032 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\code\stock-trade-system\.env\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 26448 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\code\stock-trade-system\.env\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 20998 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\code\stock-trade-system\.env\lib\site-pa